In [36]:
import os
os.environ["PATH"] = '/home/mattjones/bin:/home/mattjones/.local/bin:/home/mattjones/myapps/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/mattjones/opt/gurobi801/linux64/bin:/home/mattjones/software/bowtie2-2.3.4.2:/home/mattjones/emboss/EMBOSS-6.6.0/emboss'
os.environ["LD_LIBRARY_PATH"] = '/home/mattjones/lib/::/home/mattjones/opt/gurobi801/linux64/lib'

from ctypes import *
lib = CDLL("/home/mattjones/opt/gurobi801/linux64/lib/libgurobi80.so")


from SingleCellLineageTracing.TreeSolver import data_pipeline
from SingleCellLineageTracing.TreeSolver.simulation_tools import dataset_generation, validation
from SingleCellLineageTracing.TreeSolver.lineage_solver import solve_lineage_instance

import pandas as pd 
import numpy as np
import networkx as nx
import pickle as pic

import sys

ImportError: libgurobi80.so: cannot open shared object file: No such file or directory

In [11]:
# set parameters
D = 12
C = 40
S = 20
num_cells = 400
sub_samp = num_cells / 2**D
M = 0.0275

dropout = dict(zip(range(C), np.random.randint(7, high=40, size=C) / 100))

# create mutation map
mutation_map = {}
for i in range(C):
    sampled_probabilities = sorted([np.random.negative_binomial(5, .5) for _ in range(1, S)])
    mutation_map[i] = {'0': (1 - M)}
    total = np.sum(sampled_probabilities)
    for j in range(1, S):
        mutation_map[i][str(j)] = (M)*sampled_probabilities[j-1]/(1.0 * total)

In [13]:
true_network = dataset_generation.generate_simulated_full_tree(mutation_map, dropout, characters=C, subsample_percentage = sub_samp, depth=D)